In [ ]:
import json
import IPython
from tree import *
from load_blosum import load_blosum
f = open("organisms.json", "r")
ORGANISMS = json.load(f)
f.close()
BLOSUM50_SCORES = load_blosum("blosum50.bla")
BLOSUM62_SCORES = load_blosum("blosum62.bla")


# Reading Blosum Tables

Now that we have an alphabet for DNA strings and an algorithm to compare them, we can define meaningful costs for matchings and substitutions of amino acids. We will be using tables obtained from an experimental, data-driven technique that constructs a "Blocks Substitution Matrix," or "BLOSUM" for short, as described in [this paper](http://146.6.100.192/users/BCH339N_2018/BLOSUM62Miscalculations.pdf). In a nutshell, the technique computes statistical likelihoods that substitutions take place by examining many well-aligned sequences.

There are different similarity thresholds at which BLOSUM tables can be constructed, and higher numbers mean that we're more conservative in which symbols we allow to be aligned. In assignment 5, we'll be considering BLOSUM50 and BLOSUM62, as obtained from [this link](https://ftp.ncbi.nih.gov/repository/blocks/unix/blosum/blosum.tar.Z). Click the "show/hide" buttons to view each of these below. You'll notice that we get a positive score when matching an amino acid to itself, and a negative score when swapping amino acids and when deleting them (matching to a *), so this is compatible with Needleman-Wunsch. You'll also notice that this matrix is symmetric (e.g. A to D and D to A are both -2), so we will only be storing the non-redundant parts in a lookup dictionary. 

## Needleman-Wunsch Algorithm Background
Since DNA is a string, we can compare two DNA sequences with string comparison methods, but when comparing the sequences across organisms, the technique needs to account for mutations that have occurred over evolutionary histories, including additions, deletions, and substitutions of individual amino acids. We've discussed dynamic programing techniques for computing the string edit distance, which accounts for such mutations. However, it models a unit cost for an addition, deletion, and substitution alike, and there are biological reasons that we may want to have costs that are more less expensive for certain amino acid edits, as explained below.

To address the need for variable costs, there is a variant of edit distance known as Needleman-Wunsch, in which the costs can change depending on what characters are involved. By convention, we actually switch from a "minimizing cost" mindset to a maximizing score mindset. Given a string of length M and a string of length N, the optimal Needleman-Wunsch score can be computed in O(MN) time using a similar dynamic programming algorithm to edit distance. In particular, to match a string of length M to a string of length N, the vanilla version proceeds by filling in an (M+1) x (N+1) table S storing all solutions to sub-problems, using the following recurrence relation, where di and dj refer to the cost of deleting the ith and jth characters of the first and second strings, respectively, and cij refers to the cost of substituting the ith character of the first string for the jth character of the second string (and everything is 0-indexed):

![equation](imgs/NeedWunchEqnSmall.png)
## VERY IMPORTANT - **READ THIS**
The equation has a number of variables in it:
- _i_ and _j_ correspond directly to `i` and `j` in the code below
- _S_ is represented by the 2d list `similarity`, and is the list of all similarity scores. To access it, simply do so as a 2d list, using the comma separated values.
- _c_ and _d_ are represented within `blosum_scores`
- To access _d_, the key is simply the character you are looking to find the deletion score for. ie. if you're looking for the cost of deleting `'X'` you would simply do `blosum_scores['X']`.
    - If a term in the equation asks for _d_<sub>_i_-1</sub> or _d_<sub>_j_-1</sub>, you should use the _i_-1th or _j_-1th term of `species1` and `species2` respectively as the key 
- To access _c_, the key is the concatenation of the two characters you are looking to find the score for exchanging. ie. if you're looking for the cost of exchanging `'X'` for `'Y'` you would do either `blosum_scores['YX']` or `blosum_scores['XY']`.
    - find the characters the same way as for _d_, but get both characters and concatenate

Using these strategies, you can write the three branches of the conditional seen in the equation above. Once written it will fill in each value in the 2d array by taking into account the values above and to the left of it.

To get the final Needleman-Wunsch score, you then get the value in the bottom right corner.


In [ ]:
def needleman_wunsch(species1, species2, blosum_scores):
    """Returns the needleman wunch scores of the two strings using the provided swapping scores"""
    similarity = [[None for i in range(len(species1)+1)] for i in range(len(species2)+1)]
    for i in range(len(similarity)):
        for j in range(len(similarity[i])):
            if i == j and j == 0:
                # This is just default behavior
                similarity[i][j] = 0
            elif : # Look at the conditions on the right of the equation
                similarity[i][j] = 
            elif :
                similarity[i][j] = 
            else:
                # Remember you can use the max() function to find the maximum value among its parameteres
                similarity[i][j] = 
    
    # CHECKING IF ALL VALUES WERE REPLACED
    for i in similarity:
        if None in i:
            raise Exception("Similarity matrix is not fully filled out")
    

    score  = 

    return score

if needleman_wunsch(ORGANISMS["Dog"], ORGANISMS["Hyaena"], BLOSUM62_SCORES) != 1375:
    raise Exception(f"Dog-Hyaena similarity should be 1375, is {needleman_wunsch(ORGANISMS["Dog"], ORGANISMS["Hyaena"], BLOSUM62_SCORES)}")
elif needleman_wunsch(ORGANISMS["Domestic Cat"], ORGANISMS["Cougar"], BLOSUM62_SCORES) != 1427:
    raise Exception(f"Cat-Cougar similarity should be 1427, is {needleman_wunsch(ORGANISMS["Domestic Cat"], ORGANISMS["Cougar"], BLOSUM62_SCORES)}")
else:
    print("Dog-Hyaena similarity is "+str(needleman_wunsch(ORGANISMS["Dog"], ORGANISMS["Hyaena"], BLOSUM62_SCORES)))
    print("Cat-Cougar similarity is "+str(needleman_wunsch(ORGANISMS["Domestic Cat"], ORGANISMS["Cougar"], BLOSUM62_SCORES)))

The first `if` statement has two cases: one that runs only if `data.json` is blank, and one that runs if it has any text in it.

You do not need to change the second case. Inside the first case, you need to iterate through eah similarity pair, skipping those that are comparing an animal to itself (Hint: use `continue` in an `if` statement inside your `for` loops).

Then add to data at the keys [animal1][animal2] the needleman-wunsch BLOSUM 62 scores.

Look over your data once it's finished running (this may take 10+ minutes, plan accordingly), it'll be dumped into `data.json`.


In [ ]:
f = open('data.json', "r+") 
str(f)
data = {}
if f.read(1) == "": # If you want to rerun this part of code, manually clear the data.json file
    f.seek(0) # this resets the r/w header to the file beginning
    for i in ORGANISMS:
        data[i] = {}
    for i in ORGANISMS:
        for j in ORGANISMS:
            try:
                data[i][j] = needleman_wunsch(ORGANISMS[i], ORGANISMS[j], BLOSUM62_SCORES)
            except:
                print(i,j,data)

    f.write(json.dumps(data)) # dumps data into the file
else: 
    f.seek(0)
    data = json.loads(f.read())
f.close()

First, look at your data file. Notice that it's of the form 
```json
{
    species1: {otherspecies1:similarity,...,otherspecies70:similarity},
    ...,
    species71: {otherspecies1:similarity,...,otherspecies70:similarity}
}
```
which translates cleanly to a dictionary of dictionaries.

Create a list of all pairs of the form `[((species, species), similarity),...]` with no duplicates by iterating through the 2D dictionary you just created
<details>
<summary><strong>Click here for a hint!</strong></summary>
Try nested for loops, with the inner loop changing its range based on the outer one (you'll need a list of animals)
<details>
<summary>Click here for another hint!</summary>
<pre>
<code>
for i in range(len(animals)):
    for j in range(len(i)):
        # Do stuff

</code>
</pre>
</details>
</details>

In [ ]:
# The type after the colon is an annotation telling you what type it's intended to be!
pairs:list[tuple[tuple[str,str],float]] = 
animals:list = 

# CODE GOES HERE

pairs.sort(key=lambda x : x[1]) # This sorts by the second value in each item in pairs
# Write what part it's sorting by:
pairs.reverse()
animal = {i:Node(None, None,i,None) for i in animals} # This makes a node for each animal and keys it the animal's name


Fill in `leftanimal` and `rightanimal` inside `cluster` to access the left and right animals with each pair as you iterate.

In [ ]:
for i in pairs:
    Node.cluster(leftanimal, rightanimal)
n = animals["African bush elephant"]
while n.parent:
    n = n.parent
if len(n.all_leaves()) != len('Western honeybee|Russels viper|American alligator|African bush elephant|Guinea pig|Brown rat|House mouse|Malayan porcupine|Virginia opossum|Eastern gray kangaroo|Wallaby|Chipmunk|Fox squirrel|Wild boar|Horse|Asian black bear|American black bear|Polar bear|Giant panda|White-tailed deer|Reindeer|Northern giraffe|Domestic Yak|Cattle|Indian rhinoceros|White rhinoceros|Dog|Dingo|Indian wolf|Red fox|Hyaena|Domestic Cat|Cougar|Tiger|Ocelot|Cheetah|Eastern wolf|Dolphin|American beaver|Platypus|Koala|Orangutan|Human|Neanderthal|Chimpanzee|Bonobo|Gorilla|Gray treefrog|Edible frog|Goldfish|Fugu rubripes|Fire salamander|Eastern newt|Great white shark|Alpine newt|Chinese giant salamander|Eurasian golden oriole|Cardinal|American robin|Bald Eagle|Mourning dove|Eurasian eagle-owl|Chameleon|Bearded Dragon|Monarch butterfly|Common clothes moth|Housefly|Termite|Asian lady beetle|Black garden ant|Spotted Lanternfly'.split("|")):
    raise Exception("Not all nodes connected")
elif "|".join(n.all_leaves()) != 'Western honeybee|Russels viper|American alligator|African bush elephant|Guinea pig|Brown rat|House mouse|Malayan porcupine|Virginia opossum|Eastern gray kangaroo|Wallaby|Chipmunk|Fox squirrel|Wild boar|Horse|Asian black bear|American black bear|Polar bear|Giant panda|White-tailed deer|Reindeer|Northern giraffe|Domestic Yak|Cattle|Indian rhinoceros|White rhinoceros|Dog|Dingo|Indian wolf|Red fox|Hyaena|Domestic Cat|Cougar|Tiger|Ocelot|Cheetah|Eastern wolf|Dolphin|American beaver|Platypus|Koala|Orangutan|Human|Neanderthal|Chimpanzee|Bonobo|Gorilla|Gray treefrog|Edible frog|Goldfish|Fugu rubripes|Fire salamander|Eastern newt|Great white shark|Alpine newt|Chinese giant salamander|Eurasian golden oriole|Cardinal|American robin|Bald Eagle|Mourning dove|Eurasian eagle-owl|Chameleon|Bearded Dragon|Monarch butterfly|Common clothes moth|Housefly|Termite|Asian lady beetle|Black garden ant|Spotted Lanternfly':
    raise Exception(f"Clustering is in wrong order")

This is a simple navigation system, you can use 'up', 'left', and 'right' 
to navigate through the map. Hit enter to move on.

In [ ]:

n = animals["African bush elephant"]
while n.parent:
    n = n.parent
print("""This is a simple navigation system, you can use 'up', 'left', and 'right' 
to navigate through the map. Hit enter to move on.""")
i = "BLANK"
while i != "":
    print(n.animal, n.score)
    i = input()
    if i.lower() == "up" and n.parent:
        n = n.parent
    if i.lower() == "left" and n.left:
        n = n.left
    if i.lower() == "right" and n.right:
        n = n.right
while n.parent:
    n = n.parent
animals_leaves_list = n.all_leaves()
animals_leaves_list

You may need to run the following in yourr terminal if the next code block doesn't work:
- `pip install matplotlib`
- `pip install numpy`

If those error try:
- `pip3 install matplotlib`
- `pip3 install numpy`


### TREE OF LIFE
This generates the tree of life graphic based on your data.

Answer the questions inside the code.

In [ ]:

import matplotlib
import matplotlib.pyplot
import numpy as np

matplotlib.pyplot.figure(figsize = (10,18))
matplotlib.pyplot.xlim(800,1700)
sp = matplotlib.pyplot.subplot()
# This creates a 10 by 18 space for the graph to be placed, and limits the x axis to cover 800 through 1700 
### Based on the order of magnitude, what do you think the x-axis is?: 
# 
# 
# 

n.inorder(True)
# This runs an inorder traversal on the tree and assigns each node its order within the tree

dlist = n.points(animals_leaves_list, int(pairs[0][1])+10)
# This finds the points for the tree by recursively traversing the tree and collecting their similarity scores 
# and inorder traversal order (x and y respectively)

x = [i[0] for i in dlist]
y = [i[1] for i in dlist]
# Separates out the x and y

sp.scatter([int(pairs[0][1])+10 for i in n.all_leaves()],[2*n.all_leaves().index(i) for i in n.all_leaves()]) 
# places all the leaf nodes (the animals)
for i in range(len(n.all_leaves())):
    sp.text(pairs[0][1]+18,i*2-0.2,n.all_leaves()[i])
# labels all the animals

sp.scatter(x,y)
# 
l = n.draw_lines()
#lists all the connections
x = l[0]
y = l[1]

#What do the above lines do?:
# 
# 
#
 
lines = []
#
for i in range(len(x)):
    lines.append([(x[i][j],y[i][j]) for j in range(len(x[j]))])
for i in range(len(lines)):
    sp.plot(np.linspace(lines[i][0][0],lines[i][1][0]),np.linspace(lines[i][0][1],lines[i][1][1]))
### MAKE AN EDUCATED GUESS: What do you think linspace might do? Try printing out values
### within the code to figure it out. No Googling!
# 
# 
# 
## HINT: Plot draws a line that follows a sequence of x and y values
## https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
